## IMPORTS

In [36]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, sum,when,isnan,split,to_date,datediff,when,lit
from pyspark.ml.functions import vector_to_array
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


## INITIALIZING SESSION

In [37]:
spark = SparkSession.builder \
    .appName("Kickstarter Analysis") \
    .getOrCreate()

In [38]:
kickstarter_df = spark.read.csv('kick_starter.csv', header=True, inferSchema=True)

kickstarter_df.printSchema()


root
 |-- ID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- deadline: string (nullable = true)
 |-- goal: string (nullable = true)
 |-- launched: string (nullable = true)
 |-- pledged: string (nullable = true)
 |-- state: string (nullable = true)
 |-- backers: string (nullable = true)
 |-- country: string (nullable = true)
 |-- usd pledged: string (nullable = true)
 |-- usd_pledged_real: string (nullable = true)
 |-- usd_goal_real: string (nullable = true)



## Creating temp view

In [39]:
kickstarter_df.createOrReplaceTempView("kickstarter")


In [40]:
num_rows = kickstarter_df.count()
print(f"Number of rows: {num_rows}")

Number of rows: 378661


In [41]:
kickstarter_df_cleaned = kickstarter_df \
    .withColumn("usd_goal_real", col("usd_goal_real").cast("int")) \
    .withColumn("usd_pledged_real", col("usd_pledged_real").cast("int")) \
    .withColumn("backers", col("backers").cast("int")) \

kickstarter_df_cleaned.printSchema()


root
 |-- ID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- deadline: string (nullable = true)
 |-- goal: string (nullable = true)
 |-- launched: string (nullable = true)
 |-- pledged: string (nullable = true)
 |-- state: string (nullable = true)
 |-- backers: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- usd pledged: string (nullable = true)
 |-- usd_pledged_real: integer (nullable = true)
 |-- usd_goal_real: integer (nullable = true)



### Removing all rows with country NO".

In [42]:
kickstarter_df_cleaned = kickstarter_df_cleaned.filter(kickstarter_df.country != 'N,0"')

kickstarter_df_cleaned = kickstarter_df_cleaned.filter(kickstarter_df.country != 'N,0""')


num_rows = kickstarter_df_cleaned.count()
print(f"Number of rows: {num_rows}")



Number of rows: 374866


## Removing duplicates (No duplicates found)

In [43]:
kickstarter_df_cleaned = kickstarter_df_cleaned.dropDuplicates()
num_rows = kickstarter_df_cleaned.count()
print(f"Number of rows: {num_rows}")


Number of rows: 374866


### Upon inspection we found that the conversions done through fixer.io were more accurate and hence we decided to drop the 'usd pledged' , pledged , goal ,Name and ID as they are unecessary to our task , We will rely on all the amounts in USD to have consistency.

In [44]:
columns_to_drop = ['ID', 'goal', 'pledged', 'usd pledged','name','category']
kickstarter_df_cleaned = kickstarter_df_cleaned.drop(*columns_to_drop)

kickstarter_df_cleaned.printSchema()



root
 |-- main_category: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- deadline: string (nullable = true)
 |-- launched: string (nullable = true)
 |-- state: string (nullable = true)
 |-- backers: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- usd_pledged_real: integer (nullable = true)
 |-- usd_goal_real: integer (nullable = true)



### Removing Noisy Values in the state column (dropping 'live' as requested)

In [45]:
states_to_keep = ['successful', 'failed', 'canceled','suspended']

kickstarter_df_cleaned = kickstarter_df_cleaned.filter(col('state').isin(states_to_keep))


num_rows = kickstarter_df_cleaned.count()
print(f"Number of rows: {num_rows}")

Number of rows: 370779


### Removing the time from launched column as it is irrelevant to us.

In [46]:
date_time_split = split(col('launched'), ' ')
date_only = date_time_split.getItem(0)
date_col = to_date(date_only, 'yyyy-MM-dd')
kickstarter_df_cleaned = kickstarter_df_cleaned.withColumn('launched', date_col)

kickstarter_df_cleaned.show()




+-------------+--------+----------+----------+----------+-------+-------+----------------+-------------+
|main_category|currency|  deadline|  launched|     state|backers|country|usd_pledged_real|usd_goal_real|
+-------------+--------+----------+----------+----------+-------+-------+----------------+-------------+
| Film & Video|     USD|2012-08-10|2012-07-07|    failed|     12|     US|             296|         4000|
| Film & Video|     USD|2014-01-05|2013-11-21|successful|    148|     US|           25712|        25000|
|   Publishing|     USD|2011-05-16|2011-04-16|    failed|      0|     US|               0|          200|
|   Technology|     USD|2015-06-10|2015-05-11|successful|    298|     US|           23447|        10000|
|        Games|     GBP|2013-11-30|2013-10-31|successful|    122|     GB|            7660|         3268|
|       Comics|     NOK|2017-11-13|2017-10-20|    failed|      3|     NO|             128|         2063|
| Film & Video|     USD|2016-06-30|2016-05-16|successfu

### Created duration column which is difference between deadline and launched

In [47]:
kickstarter_df_cleaned = kickstarter_df_cleaned.withColumn("duration in days", datediff(col('deadline'), col('launched')))
kickstarter_df_cleaned.show()


+-------------+--------+----------+----------+----------+-------+-------+----------------+-------------+----------------+
|main_category|currency|  deadline|  launched|     state|backers|country|usd_pledged_real|usd_goal_real|duration in days|
+-------------+--------+----------+----------+----------+-------+-------+----------------+-------------+----------------+
| Film & Video|     USD|2012-08-10|2012-07-07|    failed|     12|     US|             296|         4000|              34|
| Film & Video|     USD|2014-01-05|2013-11-21|successful|    148|     US|           25712|        25000|              45|
|   Publishing|     USD|2011-05-16|2011-04-16|    failed|      0|     US|               0|          200|              30|
|   Technology|     USD|2015-06-10|2015-05-11|successful|    298|     US|           23447|        10000|              30|
|        Games|     GBP|2013-11-30|2013-10-31|successful|    122|     GB|            7660|         3268|              30|
|       Comics|     NOK|

### Removing unwanted main categories

In [48]:
mainCategoriesToKeep = [
  'Film & Video',
  'Music',
  'Publishing',
  'Games',
  'Technology',
  'Design',
  'Art',
  'Food',
  'Fashion',
  'Theater',
  'Comics',
  'Photography',
  'Crafts',
  'Journalism',
  'Dance'
]


kickstarter_df_cleaned = kickstarter_df_cleaned.filter(col('main_category').isin(mainCategoriesToKeep))


num_rows = kickstarter_df_cleaned.count()
print(f"Number of rows: {num_rows}")




Number of rows: 370779


### Setting successful state =1 and any other =0

In [49]:
kickstarter_df_cleaned = kickstarter_df_cleaned.withColumn(
    "state",
    when(col("state") == "successful", lit(1))  # Change 'successful' to 1
    .otherwise(lit(0))  # Change anything else to 0
)

## Looking for outliers but they represent about third of our entire dataset so we decided to include them as they may hold some reason and our ML models are not sensitive to outliers.

In [50]:
quantiles_goal = kickstarter_df_cleaned.approxQuantile("usd_goal_real", [0.25, 0.75], 0.05)
quantiles_pledged = kickstarter_df_cleaned.approxQuantile("usd_pledged_real", [0.25, 0.75], 0.05)

Q1_goal, Q3_goal = quantiles_goal
IQR_goal = Q3_goal - Q1_goal

Q1_pledged, Q3_pledged = quantiles_pledged
IQR_pledged = Q3_pledged - Q1_pledged

# Define the bounds for outliers
lower_bound_goal = Q1_goal - 1.5 * IQR_goal
upper_bound_goal = Q3_goal + 1.5 * IQR_goal

lower_bound_pledged = Q1_pledged - 1.5 * IQR_pledged
upper_bound_pledged = Q3_pledged + 1.5 * IQR_pledged

# Filter the outliers
outliers_goal = kickstarter_df_cleaned.filter((col("usd_goal_real") < lower_bound_goal) | (col("usd_goal_real") > upper_bound_goal))
outliers_pledged = kickstarter_df_cleaned.filter((col("usd_pledged_real") < lower_bound_pledged) | (col("usd_pledged_real") > upper_bound_pledged))

# Count the number of outliers
num_outliers_goal = outliers_goal.count()
num_outliers_pledged = outliers_pledged.count()

# Print the number of outliers
print(f"Number of outliers in usd_goal_real: {num_outliers_goal}")
print(f"Number of outliers in usd_pledged_real: {num_outliers_pledged}")

Number of outliers in usd_goal_real: 48092
Number of outliers in usd_pledged_real: 59358


# Querying the Data

## States Distribution (After Cleaning)

In [51]:
dist_states = kickstarter_df_cleaned.groupBy("state").agg(count("*").alias("count")).orderBy(col("count").desc())
dist_states.show()


+-----+------+
|state| count|
+-----+------+
|    0|237455|
|    1|133324|
+-----+------+



In [52]:
freq_dist_states = dist_states.toPandas()
# Create the interactive pie chart
fig = px.pie(
    freq_dist_states,
    names='state',
    values='count',
    title='Frequency of States',
    labels={'state': 'States', 'count': 'Frequency'}
)

# Show the plot
fig.show()

## Average goal per catergory (After Cleaning)

In [53]:
avg_goal_per_category = kickstarter_df_cleaned.groupBy("main_category").agg(avg("usd_goal_real").alias("avg_goal")).orderBy(col("avg_goal").desc())
avg_goal_per_category.show()


+-------------+------------------+
|main_category|          avg_goal|
+-------------+------------------+
|   Technology|102682.09718160891|
|   Journalism| 86727.19774851316|
| Film & Video| 82659.23463849115|
|         Food| 49267.37875184517|
|        Games|45147.708965912345|
|       Design|42032.050881679905|
|          Art| 39470.80283509779|
|      Theater|27346.516203918654|
|   Publishing|   25247.124040789|
|      Fashion| 22449.50541500222|
|       Comics|19692.766968537017|
|        Music|15844.886689949553|
|  Photography|12290.382825640067|
|       Crafts|10395.652722063038|
|        Dance| 9619.600805369128|
+-------------+------------------+



In [54]:
avg_goal_per_category = avg_goal_per_category.toPandas()
# Create the interactive bar chart
fig = px.bar(
    avg_goal_per_category,
    x='main_category',
    y='avg_goal',
    color='main_category',
    title='Main Category Average Goal',
    labels={'main_category': 'Main Category', 'avg_goal': 'Average Goal'},
    barmode='group'
)

# Show the plot
fig.show()

## Top 5 Total pledged by country in USD (After Cleaning)

In [55]:
total_pledged_by_country = kickstarter_df_cleaned.groupBy("country").agg(sum("usd_pledged_real").alias("total_pledged")).orderBy(col("total_pledged").desc()).limit(5)
total_pledged_by_country.show()

+-------+-------------+
|country|total_pledged|
+-------+-------------+
|     US|   2814358746|
|     GB|    235289223|
|     CA|     90717260|
|     AU|     46374209|
|     DE|     39527947|
+-------+-------------+



In [56]:
total_pledged_by_country = total_pledged_by_country

# Create the interactive pie chart
fig = px.pie(
    total_pledged_by_country,
    names='country',
    values='total_pledged',
    title='Country Total Pledged',
    labels={'country': 'Country', 'total_pledged': 'Total Pledged'}
)

# Show the plot
fig.show()

# Prepping our data to feed the model.

##  Converting to pandas df

In [57]:

kickstarter_pandas_df = kickstarter_df_cleaned.toPandas()
kickstarter_pandas_df.head()

kickstarter_pandas_df['state'].value_counts()



state
0    237455
1    133324
Name: count, dtype: int64

### Transforming data.

##### Label encoding main_category and country

In [58]:
label_encoder = LabelEncoder()

# Encode labels in column 'species'.
kickstarter_pandas_df['country']= label_encoder.fit_transform(kickstarter_pandas_df['country'])
kickstarter_pandas_df['main_category']= label_encoder.fit_transform(kickstarter_pandas_df['main_category'])

print(kickstarter_pandas_df['country'].unique())
print(kickstarter_pandas_df['main_category'].unique())



[21  9 17  1 16  7  3 12  8  5 15 19 18  4 11  6 20  0  2 10 13 14]
[ 6 12 13  8  1 10  5  0 11  7  2  4  3 14  9]


##### Label encoding the state column

In [59]:
kickstarter_pandas_df['state'] = label_encoder.fit_transform(kickstarter_pandas_df['state'])
kickstarter_pandas_df.head()

kickstarter_pandas_df['state'].value_counts()


# 2 is sucessfull and 1 is failed , 0 is canceled and 3 is suspended



state
0    237455
1    133324
Name: count, dtype: int64

In [60]:
kickstarter_pandas_df.dtypes

main_category        int32
currency            object
deadline            object
launched            object
state                int64
backers              int32
country              int32
usd_pledged_real     int32
usd_goal_real        int32
duration in days     int32
dtype: object

In [61]:
kickstarter_pandas_df.to_csv('kickstarter_cleaned.csv', index=False)
